<a href="https://colab.research.google.com/github/rgmartin/greece_tourism_project/blob/main/greece_tourism_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Greece tourism detination analysis



## 1. Web scrapping excel files

In [ ]:
! pip install plotly --upgrade

     |████████████████████████████████| 25.3 MB 1.6 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [ ]:
#####Step 1: start by importing all of the necessary packages#####
import requests #requesting URLs
import urllib.request #requesting URLs
import os
import time #setting the speed at which the requests run
import re #regexp string manipulation
import pandas as pd #for simplifying data operations (e.g. creating dataframe objects)
import matplotlib.pyplot as plt #for plotting the scraped data
from bs4 import BeautifulSoup #for web-scraping operations

In [ ]:
#####Step 2: connect to the URL in question for scraping#####
url = 'https://insete.gr/perifereies/' 
response = requests.get(url) #Connect to the URL using the "requests" package
response #if successful then it will return 200
#####Step 3: read in the URL via the "BeautifulSoup" package#####
soup = BeautifulSoup(response.text, 'html.parser')
#print(soup)
#####Step 4: filter the HTML object for all link objects######
#link_objs = soup.find_all('a', href=re.compile('/wp-content/uploads'))
excel_links = [link.get('href') for link in soup.find_all('a') if ('xlsx' in link.get('href')) ]

In [ ]:
df_parameters= [
                          {'sheet_name': 6, 'header':[2,3],'index_col':[0,1]}
]

In [ ]:
def extract_region_name(url):
  file_w_extension = os.path.basename(url)
  file = os.path.splitext(file_w_extension)[0]  
  return ''.join(x for x in file if x.isalpha())

## Origin country and destination regions for incoming tourism in Greece

In [ ]:
def read_inbound_tourism_per_country(url, key_figure = 0):
  data = requests.get(url).content
  xl = pd.ExcelFile(data)
  parameters = {'sheet_name': 2, 'header':[3],'index_col':[0,1]}
  # localise those rows corresponding to the first multindex
  df = xl.parse(**parameters)  
  region_name = extract_region_name(url)
  second_level_last_index = list(df.index.get_level_values(1)).index('Σύνολο') 
  df = df.iloc[ :second_level_last_index,key_figure].droplevel(level=0)
  #df.rename(columns={ df.columns[1]: region_name }, inplace = True)
  df.name = region_name
  return df
  
inbound_tourism_df = pd.concat([read_inbound_tourism_per_country(url) for url in excel_links], axis = 1)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
def nodes_df(df):
  regions = list(df.columns)
  countries = list(df.index)
  nodes = pd.DataFrame(data = regions + countries, columns = ['Label'])
  nodes.index.name = 'ID'

  cm = plt.get_cmap('tab20')
  n = len(regions  + countries)
  colors = [matplotlib.colors.rgb2hex(rgba) for rgba in cm(range(n))]
  nodes['Color'] = colors

  return nodes  

def links_df(nodes, input_df):

  links = input_df.stack().reset_index()
  links = links.rename(columns = {
      links.columns[0]: 'Origin',
      links.columns[1]: 'Destination',
      links.columns[2]: 'Value',
  })
  links['Origin_id'] = links.apply(lambda row: nodes.index[nodes.Label ==row['Origin']].to_list()[0],axis =1)
  links['Destination_id'] = links.apply(lambda row: nodes.index[nodes.Label ==row['Destination']].to_list()[0],axis =1)  
  links['Color'] = links.apply(lambda row: nodes.Color[nodes.Label ==row['Destination']].values[0],axis =1)  
  
  return links

nodes = nodes_df(inbound_tourism_df)
links = links_df(nodes, inbound_tourism_df)

# sort nodes first by outgoing value, then 

import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = nodes['Label'],
      color = list(nodes['Color'])
    ),
    link = dict(
      source = links['Origin_id'], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = links['Destination_id'],
      value = links['Value'],
      color = [f'rgba{tuple(matplotlib.colors.to_rgba(c)[:3] + (0.2,) )}' for c in links['Color']]

  ))])
fig.update_layout( font_size=12,
                  width=800,
    height=600,)
fig.show()

## todo change hover labels
## put appropiate title

## Change of inbound tourism shares between Greek regions for the years 2010-2020

In [ ]:
def read_inbound_tourism(url):
  data = requests.get(url).content
  xl = pd.ExcelFile(data)
  parameters = {'sheet_name': 6, 'header':[3],'index_col':[0,1]}
  df = xl.parse(**parameters)
  series = df.loc['Σύνολο'].iloc[0]
  series.name = extract_region_name(url)
  return series

inbound_tourism_df = pd.concat([read_inbound_tourism(url) for url in excel_links], axis = 1)
inbound_tourism_df.index.name = 'Year'
inbound_tourism_df.sort_values(axis=1, by=2020,ascending=False,inplace=True)
inbound_tourism_shares = inbound_tourism_df.div(inbound_tourism_df.sum(axis=1), axis=0)*100
# visualisation based on https://towardsdatascience.com/create-effective-data-visualizations-of-proportions-94b69ad34410
# ideas of visualisation https://www.politico.eu/article/europe-tourism-boom-time-overtourism-top-destinations/
import plotly.express as px
fig = px.bar(inbound_tourism_shares,x = inbound_tourism_shares.index, y =inbound_tourism_shares.columns)
fig.show()

# todo put appropiate labesl and title.

## Relevant questions
* Has the Greece tourism industry been devastated by COVID restrictions?
* How the origin country distribution was affected by COVID?
  - Plot a graph of shares of visitors from different countries (see https://www.kaggle.com/ceshine/impact-of-chinese-tourism-ban-to-taiwan)
* Analysis of seasonality
* Draw insights on how tourism has changed from different countries over time.
* Use other world datasets to draw the possible reasons/factors affecting tourism.
* domestic arrivals vs international arrivals vs road arrivals

TODO: create function that is able to read any specific data frame set from the excel documents. 
- Compare the share of tourism for each region.!

## Growth rate of tourism spending by international and domestic tourists

## Population of region vs inbound tourism

## Tourism origin country